# How to change from a IfcBuildingElementProxy to IfcWall

Here is some code that changes all elements of type [IfcBuildingElementProxy](http://www.buildingsmart-tech.org/ifc/IFC2x3/TC1/html/ifcproductextension/lexical/ifcbuildingelementproxy.htm), to an element of [IfcWall](http://www.buildingsmart-tech.org/ifc/IFC2x3/TC1/html/ifcsharedbldgelements/lexical/ifcwall.htm). 

First we create a helper function to add an object to the list of objects that a relationship object refer to. This is used to add the new object when we remove the old i.e. add the inverse attributes to the new object. 

Second we create a function that takes in a path to a file, and a path to a new file, that will change all elements of type IfcWall to IfcProduct.

**Note:** The functionality in this notebook is written for readibility, as it is heavily based on the IFC schema for IfcBuildingElementProxy and IfcWall, and handles the particularity of IFC in tht it has direct and inverse attributes. Eg. Without handling the inverse attributes, we will not be able to reference the Wall object in the spatial structure, or even relate the proper openings for voids (eg. fitting windows) to it. 

An underlying assumption in this script is that the particular IfcBuildingElementProxy elements are actually walls, only exported with a more abstract entity. 

* Getting all elements of IfcBuildingElementProxy from the file
* Iterating through all elements one by one and storing a reference to all direct and iverse attributes 
* Creating a new instance of IfcWall with the direct attributes of the IfcBuildingElementProxy object.
* Removing the old IfcBuildingElementProxy object
* Iterating through all inverse attributes (holding a relational object) and adding the new IfcWall to the relational object. 
* Store the new file. 

A great reference to do this type of script is the [creating a simple wall with property set and quantity set on the ifcopenshell academy](http://academy.ifcopenshell.org/creating-a-simple-wall-with-property-set-and-quantity-information/)

In [21]:
# Helper function to add an object to the list of objects reffered to by a #relAssosciates object. 
# This is a way of handling inverse attributes. If a wall has material, it is assigned by an inverse relationship.
# To edit this, one have to add the object in question to the list of objects that this rel associates object is 
# refering to. There are several relational objects that needs to handle following.
# This is given by the schema for an IfcBuildingElementProxy as sited below.
"""
HasAssignments	 : 	SET OF IfcRelAssigns FOR RelatedObjects;
IsDecomposedBy	 : 	SET OF IfcRelDecomposes FOR RelatingObject;
Decomposes	 : 	SET [0:1] OF IfcRelDecomposes FOR RelatedObjects;
HasAssociations	 : 	SET OF IfcRelAssociates FOR RelatedObjects;

INVERSE
IsDefinedBy	 : 	SET OF IfcRelDefines FOR RelatedObjects

INVERSE ---> Special: Provide code to handle these:
ReferencedBy	 : 	SET OF IfcRelAssignsToProduct FOR RelatingProduct;

INVERSE ---> Specials
HasStructuralMember	 : 	SET OF IfcRelConnectsStructuralElement FOR RelatingElement;
FillsVoids	 : 	SET [0:1] OF IfcRelFillsElement FOR RelatedBuildingElement;
ConnectedTo	 : 	SET OF IfcRelConnectsElements FOR RelatingElement;
HasCoverings	 : 	SET OF IfcRelCoversBldgElements FOR RelatingBuildingElement;
HasProjections	 : 	SET OF IfcRelProjectsElement FOR RelatingElement;
ReferencedInStructures	 : 	SET OF IfcRelReferencedInSpatialStructure FOR RelatedElements;
HasPorts	 : 	SET OF IfcRelConnectsPortToElement FOR RelatedElement;
HasOpenings	 : 	SET OF IfcRelVoidsElement FOR RelatingBuildingElement;
IsConnectionRealization	 : 	SET OF IfcRelConnectsWithRealizingElements FOR RealizingElements;
ProvidesBoundaries	 : 	SET OF IfcRelSpaceBoundary FOR RelatedBuildingElement;
ConnectedFrom	 : 	SET OF IfcRelConnectsElements FOR RelatedElement;
ContainedInStructure	 : 	SET [0:1] OF IfcRelContainedInSpatialStructure FOR RelatedElements;
"""

def addObjectToRelAss(object_to_add,relAss):
    if relAss.is_a("IfcRelationship"):
        if object_to_add.is_a("IfcElement"):
            if relAss.is_a("IfcRelAssignsToProduct"):
                relAss.RelatingProduct = object_to_add
            elif relAss.is_a() in ["IfcRelConnectsStructuralElement","IfcRelConnectsElements","IfcRelProjectsElement"]:
                relAss.RelatingElement = object_to_add
            elif relAss.is_a("IfcRelFillsElement"):
                relAss.RelatedBuildingElement = object_to_add
            elif relAss.is_a() in ["IfcRelCoversBldgElements","IfcRelSpaceBoundary","IfcRelVoidsElement"]:
                relAss.RelatingBuildingElement = object_to_add
            elif relAss.is_a() in ["IfcRelConnectsPortToElement","IfcRelConnectsElements"]:
                relAss.RelatedElement = object_to_add
            elif relAss.is_a() in ["IfcRelContainedInSpatialStructure","IfcRelReferencedInSpatialStructure"]:
                objectsToAdd = list(relAss.RelatedElements)
                objectsToAdd.append(object_to_add)
                relAss.RelatedElements = objectsToAdd       
            elif relAss.is_a("IfcRelConnectsWithRealizingElements"):
                objectsToAdd = list(relAss.RealizingElements)
                objectsToAdd.append(object_to_add)
                relAss.RealizingElements = objectsToAdd
            else:
                objectsToAdd = list(relAss.RelatedObjects)
                objectsToAdd.append(object_to_add)
                relAss.RelatedObjects = objectsToAdd
                

In [ ]:
## Helper function. 

In [22]:
import ifcopenshell as ios
def changeWallToProduct(ifc_in,ifc_out):
    ifc_type_in = "IfcBuildingElementProxy"
    ifc_type_out = "IfcWall"

    if isinstance(ifc_in,str) and isinstance(ifc_out,str):
        f = ios.open(ifc_in)
    
        elems_to_change = [x for x in f.by_type(ifc_type_in)]
        print("to change: {}".format(len(elems_to_change)))

        for elem in elems_to_change:
        ## Code to create an IfcObject from an IfcWall. 
        # 1. Copy all attributes of the IfcWall element
        # 2. Create a IfcProduct Element with the direct attributes of the IfcWall
        # 3. remove IfcWall element from file. 
        # 4. Handle the inverse attributes 
        # 5. store new file. 
            rel_ass_objects = []
            if elem.is_a("IfcRoot"):
                """ENTITY IfcRoot;
                GlobalId	 : 	IfcGloballyUniqueId;
                OwnerHistory	 : 	IfcOwnerHistory;
                Name	 : 	OPTIONAL IfcLabel;
                Description	 : 	OPTIONAL IfcText;"""

                guid = elem.GlobalId
                oh = elem.OwnerHistory
                name = elem.Name
                descr = elem.Description

                """INVERSE
                HasAssignments	 : 	SET OF IfcRelAssigns FOR RelatedObjects;
                IsDecomposedBy	 : 	SET OF IfcRelDecomposes FOR RelatingObject;
                Decomposes	 : 	SET [0:1] OF IfcRelDecomposes FOR RelatedObjects;
                HasAssociations"""

                hAssign = elem.HasAssignments
                if hAssign:
                    #print("Has Assignments:\n",hAssign)
                    for e in hAssign:
                        rel_ass_objects.append(e)
                isDB = elem.IsDecomposedBy
                if isDB:
                    #print("Is Decomposed By: \n",isDB)
                    for e in isDB:
                        rel_ass_objects.append(e)
                decomp = elem.Decomposes
                if decomp:
                    #print("Decomposed: \n",decomp)
                    for e in decomp:
                        rel_ass_objects.append(e)
                hAssos = elem.HasAssociations
                if hAssos:
                    #print("Has Associations: \n",hAssos)
                    for e in hAssos:
                        rel_ass_objects.append(e)

            if elem.is_a("IfcObject"):
                """ ENTITY IfcObject;
                ObjectType	 : 	OPTIONAL IfcLabel;"""
                otype = elem.ObjectType

                """INVERSE
                IsDefinedBy	 : 	SET OF IfcRelDefines FOR RelatedObjects;"""
                iDBy = elem.IsDefinedBy
                if iDBy:
                    #print("Is Defined By: \n",iDBy)
                    for e in iDBy:
                        rel_ass_objects.append(e)

            if elem.is_a("IfcProduct"):
                """ENTITY IfcProduct;
                ObjectPlacement	 : 	OPTIONAL IfcObjectPlacement;
                Representation	 : 	OPTIONAL IfcProductRepresentation;"""
                op = elem.ObjectPlacement
                repre = elem.Representation

                """INVERSE
                ReferencedBy	 : 	SET OF IfcRelAssignsToProduct FOR RelatingProduct;"""
                refBy = elem.ReferencedBy
                if (refBy):
                    #print("Is referenced by: \n",refBy)
                    for e in refBy:
                        rel_ass_objects.append(e)
            if elem.is_a("IfcElement"):
                """ENTITY IfcElement;
                Tag	 : 	OPTIONAL IfcIdentifier;"""
                tag = elem.Tag
                
                """INVERSE
                HasStructuralMember	 : 	SET OF IfcRelConnectsStructuralElement FOR RelatingElement;
                FillsVoids	 : 	SET [0:1] OF IfcRelFillsElement FOR RelatedBuildingElement;
                ConnectedTo	 : 	SET OF IfcRelConnectsElements FOR RelatingElement;
                HasCoverings	 : 	SET OF IfcRelCoversBldgElements FOR RelatingBuildingElement;
                HasProjections	 : 	SET OF IfcRelProjectsElement FOR RelatingElement;
                ReferencedInStructures	 : 	SET OF IfcRelReferencedInSpatialStructure FOR RelatedElements;
                HasPorts	 : 	SET OF IfcRelConnectsPortToElement FOR RelatedElement;
                HasOpenings	 : 	SET OF IfcRelVoidsElement FOR RelatingBuildingElement;
                IsConnectionRealization	 : 	SET OF IfcRelConnectsWithRealizingElements FOR RealizingElements;
                ProvidesBoundaries	 : 	SET OF IfcRelSpaceBoundary FOR RelatedBuildingElement;
                ConnectedFrom	 : 	SET OF IfcRelConnectsElements FOR RelatedElement;
                ContainedInStructure	 : 	SET [0:1] OF IfcRelContainedInSpatialStructure FOR RelatedElements;"""
                hSM = elem.HasStructuralMember
                if (hSM):
                    for e in hSM:
                        rel_ass_objects.append(e)
                fV = elem.FillsVoids
                if (fV):
                    for e in fV:
                        rel_ass_objects.append(e)
                cT = elem.ConnectedTo
                if (cT):
                    for e in cT:
                        rel_ass_objects.append(e)
                hC = elem.HasCoverings
                if (hC):
                    for e in hC:
                        rel_ass_objects.append(e)
                hP = elem.HasProjections
                if (hP):
                    for e in hP:
                        rel_ass_objects.append(e)
                rIS = elem.ReferencedInStructures
                if (rIS):
                    for e in rIS:
                        rel_ass_objects.append(e)
                hPorts = elem.HasPorts
                if (hPorts):
                    for e in hPorts:
                        rel_ass_objects.append(e)
                hOpenings = elem.HasOpenings
                if (hOpenings):
                    for e in hOpenings:
                        rel_ass_objects.append(e)
                iCR = elem.IsConnectionRealization
                if (iCR):
                    for e in iCR:
                        rel_ass_objects.append(e)
                pB = elem.ProvidesBoundaries
                if (pB):
                    for e in pB:
                        rel_ass_objects.append(e)
                cFrom = elem.ConnectedFrom
                if (cFrom):
                    for e in cFrom:
                        rel_ass_objects.append(e)
                cIS = elem.ContainedInStructure
                if (cIS):
                    for e in cIS:
                        rel_ass_objects.append(e)
                

            new_elem = f.createIfcWall(guid,oh,name,descr,otype,op,repre,tag)
            print(new_elem)
            f.remove(elem)

            #### write code to handle inverse attributes --> append to realassosiates relatedobject
            if len(rel_ass_objects)>0:
                print(len(rel_ass_objects))
                for rel_ass_obj in rel_ass_objects:
                    addObjectToRelAss(new_elem,rel_ass_obj)
            f.write(ifc_out)   


In [23]:
changeWallToProduct("models/11134_D_Motebello_Heistopp_Rev.ifc","models/11134_D_Motebello_Heistopp_Rev_edited.ifc")

to change: 6
#102415=IfcWall('17oKlz1VLBvx$2asaE6HDe',#4,'<Unnamed Element>',$,$,#50523,#50514,$)
3
#102416=IfcWall('0iKB6OEwv1QwmeXHgLYbtP',#4,'<Unnamed Element>',$,$,#50599,#50592,$)
3
#102417=IfcWall('2KK9lsnLf9pvSTVz_Yzg99',#4,'<Unnamed Element>',$,$,#102086,#102079,$)
3
#102418=IfcWall('1SdMb5RUnBV9YR4IcgH2Ld',#4,'<Unnamed Element>',$,$,#102141,#102134,$)
3
#102419=IfcWall('0jtV$jwQHAQgSuABhXffIz',#4,'<Unnamed Element>',$,$,#102196,#102189,$)
3
#102420=IfcWall('2i8Js8ISfCaRdW7lCmYqow',#4,'<Unnamed Element>',$,$,#102244,#102340,$)
4


In [24]:
import ifcopenshell as ios
file = ios.open("models/11134_D_Motebello_Heistopp_Rev.ifc")
proxyElems = file.by_type("IfcBuildingElementProxy")
walls = file.by_type("IfcWall")
print("number of IfcBuildingElementProxy elements is: {}".format(len(proxyElems)))
print("number of IfcWall elements is {}".format(len(walls)))

number of IfcBuildingElementProxy elements is: 6
number of IfcWall elements is 0


In [26]:
file = ios.open("models/11134_D_Motebello_Heistopp_Rev_edited.ifc")
proxyElemsNew = file.by_type("IfcBuildingElementProxy")
walls = file.by_type("IfcWall")
print("number of IfcBuildingElementProxy elements is: {}".format(len(proxyElemsNew)))
print("number of IfcWall elements is {}".format(len(walls)))

number of IfcBuildingElementProxy elements is: 0
number of IfcWall elements is 6


Function to remove an element of a particular type from a file, and store it in a new file. 

In [18]:
import ifcopenshell as ios
def removeElementsFromIFC(ifc_in,ifc_out,elementType):
    if isinstance(ifc_in,str) and isinstance(ifc_out,str):
        f = ios.open(ifc_in)
        elements_to_remove = [x for x in f.by_type(elementType)]
        print('Removing {} elements of type {}"'.format(len(elements_to_remove),elementType))
        for elem in elements_to_remove:
            f.remove(elem)
        f.write(ifc_out)
    else:
        print("please provide a valid filename")
        
removeElementsFromIFC("models/Grethes-hus-bok-2.ifc","filtered_Grethes-hus-bok-2.ifc", "IfcWall")
    

Removing 24 elements of type IfcWall"
